In [3]:
import os
import sys
import argparse
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from torch import optim
from torch.autograd import Variable
from torch.utils.data import TensorDataset, DataLoader
import random
from torch.utils import data


data = pd.read_csv("outputfile.csv") 
print(data.head(10))

X_train = []
y_train = []
X_test = []
y_test = []

# splitting train/test data
for i,row in data.iterrows():
    if data['Train/test'][i] == 0:
        X_train.append(row)
        y_train.append(data['Authors'][i])
    else:
        X_test.append(row)
        y_test.append(data['Authors'][i])

X_train = np.array(X_train)
y_train = np.array(y_train)
X_test = np.array(X_test)
y_test = np.array(y_test)

# convert train/test data to tensors
X_train = torch.from_numpy(X_train).float()
y_train = torch.from_numpy(y_train).float()


   Train/test  Authors         0         1         2         3         4  \
0         0.0        8  0.082842  0.062132  0.045165  0.041683  0.120580   
1         0.0        8  0.116337  0.132395  0.070224  0.070766  0.195577   
2         0.0       10  0.118909  0.175408  0.238420 -0.174613 -0.066054   
3         0.0        4  0.115678  0.344400 -0.248194 -0.049383 -0.036900   
4         0.0        6  0.246459  0.042580  0.079814  0.269045 -0.178284   
5         0.0        6  0.769336 -0.259384 -0.130394 -0.124862  0.032899   
6         0.0       12  0.083313  0.086944  0.037603  0.059448  0.042310   
7         0.0        1  0.067172  0.094621  0.070049  0.082810  0.181241   
8         0.0       10  0.209737  0.089271  0.113168 -0.044568 -0.023218   
9         0.0        6  0.793581 -0.276975 -0.154611 -0.164088  0.073480   

          5         6         7  ...      1890      1891      1892      1893  \
0  0.021713 -0.012578  0.165038  ... -0.006295  0.004326 -0.002024  0.002882   
1 -

In [2]:
class DocPredict(nn.Module): 
    """
    Instantiating the nn.Linear module
    """
    
    def __init__(self, input_size, num_classes):
        super(DocPredict, self).__init__()      
        self.linear = nn.Linear(input_size, num_classes)

    def forward(self, x):
        x = x.squeeze(dim=-1)
        # using a sigmoid as an activation function
        y_pred = torch.sigmoid(self.linear(x)) 
        
        return y_pred
    

class DocFFNN():
    """ Instantiating the feed forward NN)"""
    def __init__(self, X_train, y_train, epochs=5, lr=0.01):
        super(DocFFNN, self).__init__()
        self.epochs = epochs
        self.lr = lr
        
        
    def get_samples(self, X_train, y_train):
        samples = []
        self.documents = X_train
        self.authors = y_train
        
        # randomly select a document[i]
        i = np.random.randint(0, len(X_train))
 
        for i in range(i, len(X_train)-1):
            self.doc1 = self.documents[i]
            self.doc2 = self.documents[i+1]
            
            # check if documents have different authors
            if self.authors[i] == self.authors[i+1]:
                c=1
            else:
                c=0
            # append tuples representing doc1,doc2,label and return a list of samples
            samples.append([self.doc1, self.doc2, c])
        return samples        

    
    def sample_processing(self, samples):
        for i in range(len(samples)):
            sample = torch.cat((samples[i][0], samples[i][1]))
            return sample

        
    def my_model(self, input_instance):
        self.model = DocPredict(len(input_instance),1)
    
    def train(self, X_train, y_train):
        """
        The training loop.
        """
        self.my_model(X_train)

        samples = self.get_samples(X_train, y_train)
        criterion = nn.CrossEntropyLoss()
        optimizer = optim.SGD(self.model.parameters(), lr=self.lr)

        for z in range(self.epochs):
            print("Running epoch {}...".format(z))
            for i in range(len(samples)):
                # create instance
                instance = self.sample_processing(samples)
                
                # get instance label
                label = samples[i][2]
                self.my_model(instance)

                output = self.model(instance)
             
                loss = criterion(output, label)
                optimizer.zero_grad()
                loss.backward()
                optimizer.step()


            
ffnn = DocFFNN(X_train, y_train)

print(ffnn.train(X_train, y_train))



Running epoch 0...


IndexError: Dimension out of range (expected to be in range of [-1, 0], but got 1)